In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
# Be able to import from src folder
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)
import sys
sys.path.append(os.path.join(PROJ_ROOT, 'src'))

import misc.notebookHelpers

%matplotlib inline
%config InlineBackend.figure_format = "retina"

plt.rcParams['figure.figsize'] = (10, 10)

/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation


Load paths to all of the data

In [3]:
data_path = os.path.join(PROJ_ROOT, 'data/processed/ma_labeled')

In [4]:
ma_h5files = [os.path.join(root, name) 
               for root, dirs, files in os.walk(data_path)
               for name in files
               if name.endswith(('.h5'))]
ma_h5files

['/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/processed/ma_labeled/_D113026/J24BF6GI.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/processed/ma_labeled/_D113026/J24BF6GK.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/processed/ma_labeled/_D113026/J24BF6GM.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/processed/ma_labeled/_D113027/J24BF6OA.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/processed/ma_labeled/_D113027/J24BF6OC.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/processed/ma_labeled/_D113027/J24BF6OI.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/processed/ma_labeled/_D113028/J24BF71G.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/processed/ma_labeled/_D113028/J24BF71K.h5',
 '/Users/torjushaukom/Documents/

Load a single sample to check

In [5]:
ma_h5file = ma_h5files[5]

In [6]:
data = h5py.File(ma_h5file)

In [7]:
list(data.keys())

['images', 'reference']

In [8]:
data['reference']

<HDF5 dataset "reference": shape (73, 4), type "<f8">

In [9]:
points = data['reference'][:]

In [10]:
left_points = points[:, :2]
right_points = points[:, -2:]

In [11]:
points = np.concatenate((left_points[:, None, :], right_points[:, None, :]), axis=1)

In [12]:
points.shape

(73, 2, 2)

In [13]:
misc.notebookHelpers.ultraSoundAnimation(data['images'], points=points, fps=24)

In [17]:
file_path = ma_h5file.split('/')[-2] + '/' + ma_h5file.split('/')[-1]
file_path

'_D113027/J24BF6OI.h5'

In [22]:
ds_h5file = os.path.join(PROJ_ROOT, 'data/interim/ds_ma_labeled', file_path)
ds_h5file

'/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/ds_ma_labeled/_D113027/J24BF6OI.h5'

In [23]:
ds_data = h5py.File(ds_h5file)

In [25]:
video, fps = misc.video_and_fps_from_h5py(ds_data)

In [26]:
misc.notebookHelpers.ultraSoundAnimation(video, points=points, fps=fps)

In [29]:
names = []
ds_data.visit(names.append)

In [30]:
names

['TVI',
 'TVI/data',
 'TVI/dirx',
 'TVI/diry',
 'TVI/origin',
 'TVI/pixelsize',
 'TVI/times',
 'ecg',
 'ecg/ecg_data',
 'ecg/ecg_times',
 'tissue',
 'tissue/data',
 'tissue/dirx',
 'tissue/diry',
 'tissue/ds_labels',
 'tissue/origin',
 'tissue/pixelsize',
 'tissue/times']

In [31]:
new_file = os.path.join(PROJ_ROOT, 'data/interim/test.h5')

In [32]:
new_data = h5py.File(new_file, 'w')

In [33]:
for name in names:
    if len(name.split('/')) > 1:
        new_data.create_dataset(name, data=ds_data[name])

In [34]:
new_names = []
new_data.visit(new_names.append)
new_names

['TVI',
 'TVI/data',
 'TVI/dirx',
 'TVI/diry',
 'TVI/origin',
 'TVI/pixelsize',
 'TVI/times',
 'ecg',
 'ecg/ecg_data',
 'ecg/ecg_times',
 'tissue',
 'tissue/data',
 'tissue/dirx',
 'tissue/diry',
 'tissue/ds_labels',
 'tissue/origin',
 'tissue/pixelsize',
 'tissue/times']

In [35]:
new_data.create_dataset('tissue/ma_points', data=points)

<HDF5 dataset "ma_points": shape (73, 2, 2), type "<f8">

In [36]:
new_names = []
new_data.visit(new_names.append)
new_names

['TVI',
 'TVI/data',
 'TVI/dirx',
 'TVI/diry',
 'TVI/origin',
 'TVI/pixelsize',
 'TVI/times',
 'ecg',
 'ecg/ecg_data',
 'ecg/ecg_times',
 'tissue',
 'tissue/data',
 'tissue/dirx',
 'tissue/diry',
 'tissue/ds_labels',
 'tissue/ma_points',
 'tissue/origin',
 'tissue/pixelsize',
 'tissue/times']

In [37]:
new_video, new_fps = misc.video_and_fps_from_h5py(new_data)

In [38]:
misc.ultraSoundAnimation(new_video, fps=new_fps, points=new_data['tissue/ma_points'][:])